Imports

In [1]:
import numpy as np
import requests
import json
import pandas as pd
import csv
import config  # file that contains the walkscore api key


#### Getting Coordinates

Establish a max+min longitude and latitude for a grid of coordinates and desired # of points

In [2]:
# for UCSB
lon_min = -119.889848
lon_max = -119.832095
lat_min = 34.406985
lat_max = 34.429613
num_lon_points = 30
num_lat_points = 30


Create grid of lon, lan coordinates

In [3]:
lon = np.linspace(lon_min, lon_max, num_lon_points)
lat = np.linspace(lat_min, lat_max, num_lat_points)

# Save the grid of coordinates to "grid.csv"
with open("Data/grid.csv", "w") as f:
    for i in range(len(lon)):
      for j in range(len(lat)):
        f.write(f"{lon[i]},{lat[j]}\n")


#### Getting Walkscores

Load data from csv

In [ ]:
# load excel .csv with long, lat points into pd dataframe
coordinate_df = pd.read_csv('grid.csv')

# convert pd dataframe into python dictionary, with coordinates as keys
coordinate_list = coordinate_df.values.tolist()

walkscore_csv = []
points_w_error = []


Generate walkscore values for each coordinate point provided

In [ ]:
# iterate through coordinate_list (list of lists)
# coordinate_list[i] = [lon, lat, walkscore]
for i in range(len(coordinate_list)): # range(len(coordinate_list)):
  try:
  # api call to walkscore using lon and lat parameters
    url = f"https://api.walkscore.com/score?format=json&lat={coordinate_list[i][1]}&lon={coordinate_list[i][0]}&wsapikey={config.api_key}"
    response = requests.get(url)
    # json_response  = json.loads(response.content['walkscore'])
    lat = coordinate_list[i][1]
    lon = coordinate_list[i][0]
    json_response = json.loads(response.content)
    # print(json_response['walkscore'])
    walkscore = json_response['walkscore']
    print((lon, lat, walkscore))
    # add to list that will be exported into .csv
    walkscore_csv.append([lon, lat, walkscore])
  # make exception that records all the points that give an error.
  except KeyError:
    # if an error is encountered for point(s), generate a csv of those points
    # so you can generate walkscores for those
    print((coordinate_list[i][0], coordinate_list[i][1], "error"))
    walkscore_csv.append([coordinate_list[i][0], coordinate_list[i][1], "error"])
    points_w_error.append([coordinate_list[i][0], coordinate_list[i][1]])
print("Done! Check output for any errors")


Exporting .csv's

In [ ]:
# export list of points that return errors
with open('errors.csv', 'w', newline='') as csvfile:
  writer = csv.writer(csvfile)
  writer.writerow(['Longitude', 'Latitude', 'Walkscore'])
  for i in range(len(points_w_error)):
    writer.writerow([points_w_error[i][0], points_w_error[i][1]])

# export lon, lat, walkscore into csv for importing into arcgis
with open('walkscore_csv_export.csv', 'w', newline='') as csvfile:
  writer = csv.writer(csvfile)
  writer.writerow(['Longitude', 'Latitude', 'Walkscore'])
  for i in range(len(walkscore_csv)):
    writer.writerow([walkscore_csv[i][0], walkscore_csv[i][1], walkscore_csv[i][2]])
